# Graph ConvNets in PyTorch

PyTorch implementation of the NeurIPS'16 paper:
Convolutional Neural Networks on Graphs with Fast Localized Spectral Filtering
M Defferrard, X Bresson, P Vandergheynst
Advances in Neural Information Processing Systems, 3844-3852, 2016
[ArXiv preprint](https://arxiv.org/abs/1606.09375)

Adapted from Xavier Bresson's repo: [spectral_graph_convnets](https://github.com/xbresson/spectral_graph_convnets) for [dataflowr](https://dataflowr.github.io/website/) by [Marc Lelarge](https://www.di.ens.fr/~lelarge/)

## objective:

The code provides a simple example of graph ConvNets for the MNIST classification task.
The graph is a 8-nearest neighbor graph of a 2D grid.
The signals on graph are the MNIST images vectorized as $28^2 \times 1$ vectors.

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import collections
import time
import numpy as np
import scipy
from functools import partial
import os

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    torch.manual_seed(1)

cuda available


## Download the data

If you are running on colab, follow the instructions below.

If you cloned the repo, go directly to the tempory hack.

### Colab setting

If you run this notebook on colab, please uncomment (and run) the following cells.

In [2]:
!wget www.di.ens.fr/~lelarge/graphs.tar.gz

--2021-03-16 16:12:47--  http://www.di.ens.fr/~lelarge/graphs.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/graphs.tar.gz [following]
--2021-03-16 16:12:47--  https://www.di.ens.fr/~lelarge/graphs.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘graphs.tar.gz’

graphs.tar.gz           [            <=>     ]  66.42M  17.8MB/s    in 4.9s    

2021-03-16 16:12:53 (13.7 MB/s) - ‘graphs.tar.gz’ saved [69647028]



In [3]:
!tar -zxvf graphs.tar.gz

graphs/
graphs/lib/
graphs/lib/grid_graph.py
graphs/lib/coarsening.py
graphs/.spectral_gnn.ipynb.swp
graphs/mnist/
graphs/mnist/temp/
graphs/mnist/temp/MNIST/
graphs/mnist/temp/MNIST/raw/
graphs/mnist/temp/MNIST/raw/t10k-images-idx3-ubyte
graphs/mnist/temp/MNIST/raw/t10k-labels-idx1-ubyte
graphs/mnist/temp/MNIST/raw/t10k-images-idx3-ubyte.gz
graphs/mnist/temp/MNIST/raw/t10k-labels-idx1-ubyte.gz
graphs/mnist/temp/MNIST/raw/train-images-idx3-ubyte
graphs/mnist/temp/MNIST/raw/train-images-idx3-ubyte.gz
graphs/mnist/temp/MNIST/raw/train-labels-idx1-ubyte
graphs/mnist/temp/MNIST/raw/train-labels-idx1-ubyte.gz
graphs/mnist/temp/MNIST/processed/
graphs/mnist/temp/MNIST/processed/test.pt
graphs/mnist/temp/MNIST/processed/training.pt
graphs/mnist/temp/MNIST.tar.gz
graphs/spectral_gnn.ipynb


In [4]:
%cd graphs

/content/graphs


### temporary hack 

Unecessary if running on colab (or if you already have MNIST), see this [issue](https://github.com/pytorch/vision/issues/3497)

!mkdir mnist
%cd mnist
!mkdir temp
%cd temp
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz

!tar -zxvf MNIST.tar.gz

%cd ..
%cd ..

## Loading the data

In [5]:
def check_mnist_dataset_exists(path_data='./'):
    flag_train_data = os.path.isfile(path_data + 'mnist/train_data.pt') 
    flag_train_label = os.path.isfile(path_data + 'mnist/train_label.pt') 
    flag_test_data = os.path.isfile(path_data + 'mnist/test_data.pt') 
    flag_test_label = os.path.isfile(path_data + 'mnist/test_label.pt') 
    if flag_train_data==False or flag_train_label==False or flag_test_data==False or flag_test_label==False:
        print('MNIST dataset preprocessing...')
        import torchvision
        import torchvision.transforms as transforms
        trainset = torchvision.datasets.MNIST(root=path_data + 'mnist/temp', train=True,
                                                download=True, transform=transforms.ToTensor())
        testset = torchvision.datasets.MNIST(root=path_data + 'mnist/temp', train=False,
                                               download=True, transform=transforms.ToTensor())
        train_data=torch.Tensor(60000,28,28)
        train_label=torch.LongTensor(60000)
        for idx , example in enumerate(trainset):
            train_data[idx]=example[0].squeeze()
            train_label[idx]=example[1]
        torch.save(train_data,path_data + 'mnist/train_data.pt')
        torch.save(train_label,path_data + 'mnist/train_label.pt')
        test_data=torch.Tensor(10000,28,28)
        test_label=torch.LongTensor(10000)
        for idx , example in enumerate(testset):
            test_data[idx]=example[0].squeeze()
            test_label[idx]=example[1]
        torch.save(test_data,path_data + 'mnist/test_data.pt')
        torch.save(test_label,path_data + 'mnist/test_label.pt')
    return path_data


_ = check_mnist_dataset_exists()

MNIST dataset preprocessing...


In [6]:
#if you want to play with a small dataset (for cpu), uncomment.
#nb_selected_train_data = 500
#nb_selected_test_data = 100

train_data=torch.load('mnist/train_data.pt').reshape(60000,784).numpy()
#train_data = train_data[:nb_selected_train_data,:]
print(train_data.shape)

train_labels=torch.load('mnist/train_label.pt').numpy()
#train_labels = train_labels[:nb_selected_train_data]
print(train_labels.shape)

test_data=torch.load('mnist/test_data.pt').reshape(10000,784).numpy()
#test_data = test_data[:nb_selected_test_data,:]
print(test_data.shape)

test_labels=torch.load('mnist/test_label.pt').numpy()
#test_labels = test_labels[:nb_selected_test_data]
print(test_labels.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [7]:
from lib.grid_graph import grid_graph
from lib.coarsening import coarsen, HEM, compute_perm, perm_adjacency
from lib.coarsening import perm_data

# Construct graph
t_start = time.time()
grid_side = 28
number_edges = 8
metric = 'euclidean'


######## YOUR GRAPH ADJACENCY MATRIX HERE ########
A = grid_graph(grid_side,number_edges,metric) # create graph of Euclidean grid
######## YOUR GRAPH ADJACENCY MATRIX HERE ########

nb edges:  6396


In [8]:
def laplacian(W, normalized=True):
    """Return graph Laplacian"""
    I = scipy.sparse.identity(W.shape[0], dtype=W.dtype)

    W += I
    # Degree matrix.
    d = W.sum(axis=0)

    # Laplacian matrix.
    if not normalized:
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        L = D - W
    else:
        #
        #
        # your code here for normalized laplacian
        #
        # Using Symmetric normalized Laplacian (eigenvalues <= 2)
        D_inv_sqrt = scipy.sparse.diags(1 / np.sqrt(d.A.squeeze()), 0)
        L = I - D_inv_sqrt @ W @ D_inv_sqrt

    assert np.abs(L - L.T).mean() < 1e-8
    assert type(L) is scipy.sparse.csr.csr_matrix
    return L

In [9]:
def rescale_L(L, lmax=2):
    """Rescale Laplacian eigenvalues to [-1,1]"""
    M, M = L.shape
    I = scipy.sparse.identity(M, format='csr', dtype=L.dtype)
    L = 2*L/lmax - I
    return L

def lmax_L(L):
    """Compute largest Laplacian eigenvalue"""
    return scipy.sparse.linalg.eigsh(L, k=1, which='LM', return_eigenvectors=False)[0]

In [10]:
normalized = True

# Compute coarsened graphs
coarsening_levels = 4

L, perm = coarsen(A, coarsening_levels, partial(laplacian, normalized=normalized))

# Compute max eigenvalue of graph Laplacians
lmax = []
for i in range(coarsening_levels+1):
    lmax.append(lmax_L(L[i]))
print('lmax: ' + str([lmax[i] for i in range(coarsening_levels+1)]))

# Reindex nodes to satisfy a binary tree structure
train_data = perm_data(train_data, perm)
test_data = perm_data(test_data, perm)

print('Execution time: {:.2f}s'.format(time.time() - t_start))
del perm

Heavy Edge Matching coarsening with Xavier version
Layer 0: M_0 = |V| = 960 nodes (176 added), |E| = 3198 edges
Layer 1: M_1 = |V| = 480 nodes (77 added), |E| = 1618 edges
Layer 2: M_2 = |V| = 240 nodes (29 added), |E| = 781 edges
Layer 3: M_3 = |V| = 120 nodes (7 added), |E| = 388 edges
Layer 4: M_4 = |V| = 60 nodes (0 added), |E| = 194 edges
lmax: [1.1200929, 1.1981337, 1.1376913, 0.9334328, 0.7780063]
Execution time: 1.52s


Here, we implemented the pooling layers and computed the list `L` containing the Laplacians of the graphs for each layer.

## <font color='red'>Question 1: what is the size of the various poolings?</font> 

Each pooling is of size two. We have therefore built the graph for pooling of size 0, 2, 4, 8 and 16.

# Graph ConvNet LeNet5

## Layers: CL32-MP4-CL64-MP4-FC512-FC10

As described above, this network has 2 graph convolutional layers and two pooling layers with size 4.

## <font color='red'>Question 2: which graphs will you take in the list `L` for the graph convolutional layers?</font> 



First we have to use the original Laplacian L[0].

After each max-pooling operation of size 4, we have to use L[i +2] instead of L[i]. We will therefore use L[0], L[2] and L[4] (There are 2 max pooling operations). But as there are no more convolutionnal layers after the last max pooling, L[4] is not needed and only L[0] and L[2] will be used.

In the code below, you will need to complete the `graph_conv_cheby` and the `graph_max_pool`.

Hint: each time you permute dimenstions, it is safe to add a `contiguous` like below:
`x0 = x.permute(1,2,0).contiguous()` see [here](https://discuss.pytorch.org/t/call-contiguous-after-every-permute-call/13190/2) 

In [11]:
class Graph_ConvNet_LeNet5(nn.Module):

    def __init__(self, net_parameters):

        print('Graph ConvNet: LeNet5')

        super(Graph_ConvNet_LeNet5, self).__init__()
        
        # parameters
        D, CL1_F, CL1_K, CL2_F, CL2_K, FC1_F, FC2_F = net_parameters
        FC1Fin = CL2_F*(D//16)

        # graph CL1
        self.cl1 = nn.Linear(CL1_K, CL1_F)
        self.init_layers(self.cl1, CL1_K, CL1_F)
        self.CL1_K = CL1_K; self.CL1_F = CL1_F;

        # graph CL2
        self.cl2 = nn.Linear(CL2_K*CL1_F, CL2_F)
        self.init_layers(self.cl2, CL2_K*CL1_F, CL2_F)
        self.CL2_K = CL2_K; self.CL2_F = CL2_F;

        # FC1
        self.fc1 = nn.Linear(FC1Fin, FC1_F) 
        self.init_layers(self.fc1, FC1Fin, FC1_F)
        self.FC1Fin = FC1Fin

        # FC2
        self.fc2 = nn.Linear(FC1_F, FC2_F)
        self.init_layers(self.fc2, FC1_F, FC2_F)

        # nb of parameters
        nb_param = CL1_K* CL1_F + CL1_F          # CL1
        nb_param += CL2_K* CL1_F* CL2_F + CL2_F  # CL2
        nb_param += FC1Fin* FC1_F + FC1_F        # FC1
        nb_param += FC1_F* FC2_F + FC2_F         # FC2
        print('nb of parameters=',nb_param,'\n')
        
        
    def init_layers(self, W, Fin, Fout):

        scale = np.sqrt( 2.0/ (Fin+Fout) )
        W.weight.data.uniform_(-scale, scale)
        W.bias.data.fill_(0.0)

        return W
        
        
    def graph_conv_cheby(self, x, cl, L, lmax, Fout, K):
        # parameters
        # B = batch size
        # V = nb vertices
        # Fin = nb input features
        # Fout = nb output features
        # K = Chebyshev order & support size
        B, V, Fin = x.size(); B, V, Fin = int(B), int(V), int(Fin) 

        # rescale Laplacian
        if normalized:
          # L = rescale_L(L)  # = L - I
          L = L / 2
        else:
          # lmax = lmax_L(L)  # Useless
          L = rescale_L(L, lmax)

        # convert scipy sparse matric L to pytorch
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col)).T 
        indices = indices.astype(np.int64)
        indices = torch.from_numpy(indices)
        indices = indices.type(torch.LongTensor)
        L_data = L.data.astype(np.float32)
        L_data = torch.from_numpy(L_data) 
        L_data = L_data.type(torch.FloatTensor)
        L = torch.sparse.FloatTensor(indices, L_data, torch.Size(L.shape))
        L.requires_grad_(False)
        if torch.cuda.is_available():
            L = L.cuda()

        # transform to Chebyshev basis
        # Due to limitation of sparse matrix autograd, have to reorder each input for the matrix product.
        x_0 = x.permute(1, 0, 2).contiguous()  # V x B x Fin
        x_0 = x_0.view(V, B*Fin)  # V x (B*Fin)
        x = x_0.unsqueeze(0)  # 1 x V x (B*Fin)

        if K > 1:
          x_1 = torch.sparse.mm(L, x_0)  # V x (B*Fin)
          x = torch.cat((x, x_1.unsqueeze(0)))  # 2 x V x (B*Fin)

          for k in range(2, K):
              x_2 = 2 * torch.sparse.mm(L, x_1) - x_0  # V x (B*Fin)
              x = torch.cat((x, x_2.unsqueeze(0)))  # (k+1) x V x (B*Fin)
              x_0, x_1 = x_1, x_2

        x = x.view(K, V, B, Fin)  # K x V x B x Fin
        x = x.permute(2, 1, 0, 3).contiguous()  # B x V x K x Fin
        x = x.view(B, V, K*Fin)  # B x V x K*Fin
        x = cl(x)  # B x V x Fout

        return x

    # Max pooling of size p. Must be a power of 2.
    def graph_max_pool(self, x, p): 
        # 
        # your code here
        # input B x V x F output B x V/p x F
        #
        max_pool = nn.MaxPool1d(p)
        x = max_pool(x.permute(0, 2, 1).contiguous())
        return x.permute(0, 2, 1).contiguous()

    def forward(self, x, d, L, lmax):
        # graph CL1
        x = x.unsqueeze(2) # B x V x Fin=1  
        x = self.graph_conv_cheby(x, self.cl1, L[0], lmax[0], self.CL1_F, self.CL1_K)
        x = F.relu(x)
        x = self.graph_max_pool(x, 4)
        # graph CL2
        x = self.graph_conv_cheby(x, self.cl2, L[2], lmax[2], self.CL2_F, self.CL2_K)
        x = F.relu(x)
        x = self.graph_max_pool(x, 4)
        # FC1
        x = x.view(-1, self.FC1Fin)
        x = self.fc1(x)
        x = F.relu(x)
        x  = nn.Dropout(d)(x)
        # FC2
        x = self.fc2(x)
        return x
        
        
    def loss(self, y, y_target, l2_regularization):
    
        loss = nn.CrossEntropyLoss()(y,y_target)

        l2_loss = 0.0
        for param in self.parameters():
            data = param* param
            l2_loss += data.sum()
           
        loss += 0.5* l2_regularization* l2_loss
            
        return loss
    
    
    def update(self, lr):
                
        update = torch.optim.SGD( self.parameters(), lr=lr, momentum=0.9 )
        
        return update
        
        
    def update_learning_rate(self, optimizer, lr):
   
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        return optimizer

    
    def evaluation(self, y_predicted, test_l):
    
        _, class_predicted = torch.max(y_predicted.data, 1)
        return 100.0* (class_predicted == test_l).sum()/ y_predicted.size(0)

In [12]:
# Delete existing network if exists
try:
    del net
    print('Delete existing network\n')
except NameError:
    print('No existing network to delete\n')

# network parameters
D = train_data.shape[1]
CL1_F = 32
CL1_K = 25
CL2_F = 64
CL2_K = 25
FC1_F = 512
FC2_F = 10
net_parameters = [D, CL1_F, CL1_K, CL2_F, CL2_K, FC1_F, FC2_F]
dropout_value = 0.5

# instantiate the object net of the class 
net = Graph_ConvNet_LeNet5(net_parameters)
if torch.cuda.is_available():
    net.cuda()
print(net)

No existing network to delete

Graph ConvNet: LeNet5
nb of parameters= 2023818 

Graph_ConvNet_LeNet5(
  (cl1): Linear(in_features=25, out_features=32, bias=True)
  (cl2): Linear(in_features=800, out_features=64, bias=True)
  (fc1): Linear(in_features=3840, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


Good time, to check your network is working...

In [13]:
train_x, train_y = train_data[:5,:], train_labels[:5]
train_x =  torch.FloatTensor(train_x).type(dtypeFloat)
train_y = train_y.astype(np.int64)
train_y = torch.LongTensor(train_y).type(dtypeLong) 
            
# Forward 
y = net(train_x, dropout_value, L, lmax)
print(y.shape)

torch.Size([5, 10])


In [14]:
# Weights
L_net = list(net.parameters())

# learning parameters
learning_rate = 0.05
l2_regularization = 5e-4 
batch_size = 100
num_epochs = 3
train_size = train_data.shape[0]
nb_iter = int(num_epochs * train_size) // batch_size
print('num_epochs=',num_epochs,', train_size=',train_size,', nb_iter=',nb_iter)

# Optimizer
global_lr = learning_rate
global_step = 0
decay = 0.95
decay_steps = train_size
lr = learning_rate
optimizer = net.update(lr) 

# loop over epochs
indices = collections.deque()
for epoch in range(num_epochs):  # loop over the dataset multiple times

    # reshuffle 
    indices.extend(np.random.permutation(train_size)) # rand permutation
    
    # reset time
    t_start = time.time()
    
    # extract batches
    running_loss = 0.0
    running_accuray = 0
    running_total = 0
    while len(indices) >= batch_size:
        
        # extract batches
        batch_idx = [indices.popleft() for i in range(batch_size)]
        train_x, train_y = train_data[batch_idx,:], train_labels[batch_idx]
        train_x =  torch.FloatTensor(train_x).type(dtypeFloat)
        train_y = train_y.astype(np.int64)
        train_y = torch.LongTensor(train_y).type(dtypeLong) 
            
        # Forward 
        y = net(train_x, dropout_value, L, lmax)
        loss = net.loss(y,train_y,l2_regularization) 
        loss_train = loss.detach().item()
        # Accuracy
        acc_train = net.evaluation(y,train_y.data)
        # backward
        loss.backward()
        # Update 
        global_step += batch_size # to update learning rate
        optimizer.step()
        optimizer.zero_grad()
        # loss, accuracy
        running_loss += loss_train
        running_accuray += acc_train
        running_total += 1
        # print        
        if not running_total%100: # print every x mini-batches
            print('epoch= %d, i= %4d, loss(batch)= %.4f, accuray(batch)= %.2f' % (epoch+1, running_total, loss_train, acc_train))
          
    # print 
    t_stop = time.time() - t_start
    print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f, time= %.3f, lr= %.5f' % 
          (epoch+1, running_loss/running_total, running_accuray/running_total, t_stop, lr))
 
    # update learning rate 
    lr = global_lr * pow( decay , float(global_step// decay_steps) )
    optimizer = net.update_learning_rate(optimizer, lr)
    
    
    # Test set
    with torch.no_grad():
        running_accuray_test = 0
        running_total_test = 0
        indices_test = collections.deque()
        indices_test.extend(range(test_data.shape[0]))
        t_start_test = time.time()
        while len(indices_test) >= batch_size:
            batch_idx_test = [indices_test.popleft() for i in range(batch_size)]
            test_x, test_y = test_data[batch_idx_test,:], test_labels[batch_idx_test]
            test_x = torch.FloatTensor(test_x).type(dtypeFloat)
            y = net(test_x, 0.0, L, lmax) 
            test_y = test_y.astype(np.int64)
            test_y = torch.LongTensor(test_y).type(dtypeLong)
            acc_test = net.evaluation(y,test_y.data)
            running_accuray_test += acc_test
            running_total_test += 1
        t_stop_test = time.time() - t_start_test
        print('  accuracy(test) = %.3f %%, time= %.3f' % (running_accuray_test / running_total_test, t_stop_test))

num_epochs= 3 , train_size= 60000 , nb_iter= 1800
epoch= 1, i=  100, loss(batch)= 0.3953, accuray(batch)= 93.00
epoch= 1, i=  200, loss(batch)= 0.3482, accuray(batch)= 94.00
epoch= 1, i=  300, loss(batch)= 0.1989, accuray(batch)= 96.00
epoch= 1, i=  400, loss(batch)= 0.2671, accuray(batch)= 95.00
epoch= 1, i=  500, loss(batch)= 0.2076, accuray(batch)= 95.00
epoch= 1, i=  600, loss(batch)= 0.2525, accuray(batch)= 96.00
epoch= 1, loss(train)= 0.396, accuracy(train)= 90.837, time= 29.317, lr= 0.05000
  accuracy(test) = 97.560 %, time= 3.259
epoch= 2, i=  100, loss(batch)= 0.1732, accuray(batch)= 99.00
epoch= 2, i=  200, loss(batch)= 0.2309, accuray(batch)= 95.00
epoch= 2, i=  300, loss(batch)= 0.2558, accuray(batch)= 96.00
epoch= 2, i=  400, loss(batch)= 0.2810, accuray(batch)= 95.00
epoch= 2, i=  500, loss(batch)= 0.2127, accuray(batch)= 95.00
epoch= 2, i=  600, loss(batch)= 0.1740, accuray(batch)= 97.00
epoch= 2, loss(train)= 0.195, accuracy(train)= 97.368, time= 29.140, lr= 0.04750
  a

### <font color='red'>Question 3: In this code, each convolutional layer has a parameter K. What does it represent? What are the consequences of choosing a higher or lower value of K? </font> 

K correspond to the number of degree that we will use for our polynomial basis.

It also represent the maximum distance of the neighbours that are used to compute the new embedding at each location. (as $L^k_{ij} = 0$ when there is no path of length smaller than k). The filter is K-localized. 

### <font color='red'>Question 4: Is it necessary to rescale the Laplacian (in the function `rescale_L`)? Try to remove it and explain what happens. </font> 

Hint: See Section 2.1 of [the paper](https://arxiv.org/pdf/1606.09375.pdf).


Without scaling, we have numerical issues, as the eigenvalues are not scaled, we face exploding computations.

### <font color='red'>Question 5: Is it possible to modify the Laplacian to avoid the rescaling step?</font> 

Hint: Think about the eigenvalues of the Laplacian and how to normalize them.

We can use the Symmetric Normalized Laplacian, as it has the property to have its eigenvalues between 0 and 2.

Then the rescaling step can just be a division by 2 (or substracting I) which leads to a matrix with the same eigenvectors, but eigenvalues in [0, 1] ([-1, 1] resp.). There is no more needs to compute lmax.

Note: Even with the unnormalized laplacian, lmax can be computed out of the model forward step (which was partially done here). And thus it's not a real issue to rescale.

The variable `normalized` can be used to parametrize the code in order to use the Symmetric laplacian or not.

### <font color='red'>Question 6: Is GCN like the one presented in video 2 a MGNN? </font> 
* (A) Yes for K=1
* (B) Yes for any value of K
* (C) No

Explain your answer.



GCN for K=1 can be seen directly as a MGNN. As it will compute the next features given the feature of the direct neighbours. (as MGNN)

But even for greater value of K, a one layer GCN can be seen as a multi layer MGNN. (In a K-layer MGNN, the feature is computed with the information of the K-distant neighbours, as in a K-GCN)

### <font color='red'> Question 7: In which cases do you expect: </font> 
* a Graph CNN to work better than a CNN?
* a CNN to work better than a Graph CNN?

For the MNIST classification problem, is there an advantage in using a Graph CNN instead of CNN ? Explain. 

Graph Convolution allows to generalize Convolution layer. On image it seems nonetheless that classical convolution is still better and faster (as shown in the paper).

One idea is that there is no notion of up/down with a grid graph (non oriented) representing the image. Which can be a limitation for some images.

On more complexe structure such as text, having a Graph Convolution should perform better than classical Convolution as it is able to represent more complex dependencies.

On the MNIST classification problem I don't see any advantage of GCN over CNN, as showed in the paper they have slightly worse results, and it's much slower (around 7 times slower). Nonetheless it's useful to show that it works almost as good as CNN in a simple case.